In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from evaluate_data import read_df, get_spec
from dynamic_analysis import add_pose_to_df

#exp_name = '2020_10_30_dynamic'; degree=0; start_idx = 0# correct mic positions, but only one degree
#exp_name = '2020_10_30_dynamic_test'; degree=0; start_idx=0 #degree=90; 
#exp_name = '2020_10_30_dynamic_move'; degree=0; start_idx=0
#exp_name = '2020_11_03_sweep_old'; degree=90; start_idx = 20
#exp_name = '2020_11_03_sweep'; degree=90; start_idx = 20

exp_name = '2020_11_10_buzzer'; degree=45; start_idx = 20
#exp_name = '2020_11_10_buzzer'; degree=0; start_idx = 20
#exp_name = '2020_11_10_buzzer'; degree=0; start_idx = 20

params = dict(
    degree = degree,
    props = False,
    snr = False,
    motors = True,
    source = None,
    exp_name = exp_name
)

df, df_pos = read_df(**params)
df.head()

df = df.iloc[start_idx:]
index_start = df.iloc[0]['index']
df_pos = df_pos.loc[df_pos.index >= index_start]
add_pose_to_df(df, df_pos)
df.head()

plt.figure()
i = 0
for mic in df.iloc[0].mic_positions:
    plt.scatter(mic[0], mic[1], label=f'mic{i}') 
    i += 1
plt.legend()

In [ ]:
start_pos = np.array([0, 0])
positions = np.empty([len(df), 2])

plt.figure()
i_row = 0
for i, row in df.iterrows():
    yaw_rad = row.yaw_deg / 180 * np.pi
    length = np.sqrt(row.dx**2 + row.dy**2) 
    new_pos = start_pos + length * np.array([np.cos(yaw_rad), np.sin(yaw_rad)])
    positions[i_row, :] = new_pos
    i_row += 1
    plt.scatter(row.timestamp/1000-df.timestamp.iloc[0]/1000, row.yaw_deg, color='C0')
plt.xlabel('time [s]')
plt.ylabel('yaw [deg]')
    
plt.figure()
plt.scatter(positions[:, 0], positions[:, 1])
plt.axis('equal')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')

In [ ]:
from audio_stack.beam_former import BeamFormer, normalize_rows, combine_rows
import progressbar

combination_n = 1
combination_method = "sum"
normalization_method = "none"
method = "das"
bin_selection = "fixed"; selected_hz = [4100]

averaging = 'ma'

#columns = range(len(df))
columns = list(range(len(df)))[::3]

selected_idx = [np.argmin(np.abs(df.iloc[0].frequencies - hz)) for hz in selected_hz]

# covariance averaging
#averaging = 'iir'
#alpha = 0.5 # set to 1 for no effect

In [ ]:
fig, ax = plt.subplots()
times = (df.iloc[columns].audio_timestamp - df.iloc[0].audio_timestamp) / 1e6
signals_f = np.array([*df.iloc[columns]['signals_f']]) # n_times x n_mics x n_freqs
for i in range(signals_f.shape[1]):
    ax.plot(times, 
            np.sum(np.abs(signals_f[:, i, selected_idx]), axis=1))
ax.set_ylim([1e-3, 2])
ax.legend([f"mic{i}" for i in range(row.signals_f.shape[0])])
ax.set_xlabel('time [s]')

# test different mic combinations

In [ ]:
def plot_heatmap(ax):
    extra_kwargs = dict(
        color='red',
        lw=2
    )
    angles = BeamFormer.theta_scan * 180 / np.pi

    start_orientation = orientations[~np.isnan(orientations)][0]
    raw_orientations =  start_orientation - orientations
    if degree != 45:
        raw_orientations += degree
        raw_orientations[raw_orientations < 0] += 360

    ax.pcolormesh(times, angles, raw_heatmap)
    ax.plot(times, raw_orientations, **extra_kwargs)
    ax.set_ylim(0, 360)
    ax.set_xlabel('time [s]')
    ax.set_ylabel('angle [deg]')

In [ ]:
#mic_selection = [1, 2] #[1, 3] #list(range(4))
import itertools

n_columns = len(columns)

for num in [2, 3, 4]:
    
    combis = list(itertools.combinations(range(4), num))
    n_combis = len(combis)
    
    fig, axs = plt.subplots(1, n_combis, sharey=True, sharex=True, squeeze=False)
    axs = axs.reshape((n_combis,))
    fig.suptitle(f"motors: {params['motors']}, degree: {params['degree']}")
    fig.set_size_inches(15, 5)
    axs_i = 0

    for mic_selection in combis:
        mic_positions = df.iloc[0].mic_positions[mic_selection, :]
        beam_former = BeamFormer(mic_positions=mic_positions)

        R_idx = 0; R_list = [[]] * combination_n
        R = None
    
        angles = beam_former.theta_scan
        times = np.empty(n_columns)
        orientations = np.full(n_columns, np.nan)
        raw_heatmap = np.zeros((len(angles), n_columns))

        i_col = 0
        dynamic_spectrum = None
        multi_spectrum = None

        with progressbar.ProgressBar(max_value=n_columns, redirect_stdout=True) as p:
            for i in columns:
                row = df.iloc[i]

                if bin_selection == "fixed":
                    signals_f = row.signals_f[:, selected_idx] # n_mics x n_freqs
                    signals_f = signals_f[mic_selection, :]
                    freqs = row.frequencies[selected_idx]
                else:
                    # TODO(FD): implement this
                    pass

                orientation_deg = row.yaw_deg
                timestamp = row.timestamp

                R_new = beam_former.get_correlation(signals_f.T) # n_freqs x n_mics x n_mics
                if averaging == 'iir':
                    if R is None:
                        R = R_new
                    else:
                        R = alpha*R_new + (1-alpha)*R
                elif averaging == 'ma':
                    R_list[R_idx] = R_new
                    R_idx = (R_idx + 1) % combination_n
                    R = np.mean([R_elem for R_elem in R_list if len(R_elem)], axis=0)

                if method == 'mvdr':
                    raw_spectrum = beam_former.get_mvdr_spectrum(R, freqs)
                elif method == 'das':
                    raw_spectrum = beam_former.get_das_spectrum(R, freqs)

                time_sec = row.audio_timestamp / 1e6

                #### RAW
                raw_spectrum = combine_rows(raw_spectrum, combination_method, keepdims=True)
                raw_spectrum = normalize_rows(raw_spectrum, method="zero_to_one")

                raw_heatmap[:, i_col] = raw_spectrum.flatten()
                if dynamic_spectrum is not None:
                    dynamic_heatmap[:, i_col] = dynamic_spectrum.flatten()
                if multi_spectrum is not None:
                    multi_heatmap[:, i_col] = multi_spectrum.flatten()
                times[i_col] = time_sec
                orientations[i_col] = orientation_deg
                i_col += 1

                if i_col >= n_columns:
                    break

                p.update(i_col)

        plot_heatmap(axs[axs_i])
        axs[axs_i].set_title(mic_selection)
        axs_i += 1

    #fig.savefig('figures/{exp_name}_mic_selection.png')

# test different dynamic schemes

In [ ]:
def plot_results(fig, axs):
    axs = axs.reshape((1, 3))
    fig.set_size_inches(15, 5)

    extra_kwargs = dict(
    color='red',
    lw=2
    )

    times = (df.iloc[columns].audio_timestamp - df.iloc[0].audio_timestamp) / 1e6
    angles = BeamFormer.theta_scan * 180 / np.pi

    start_orientation = orientations[~np.isnan(orientations)][0]
    raw_orientations =  start_orientation - orientations
    abs_orientations = np.full(orientations.shape, start_orientation)

    if degree != 45:
        raw_orientations += degree
        abs_orientations += degree
        raw_orientations[raw_orientations < 0] += 360
        abs_orientations[abs_orientations < 0] += 360

    axs[0, 0].pcolormesh(times, angles, raw_heatmap)
    axs[0, 0].plot(times, raw_orientations, **extra_kwargs)
    axs[0, 0].set_title('raw heatmap')

    axs[0, 1].pcolormesh(times, angles, multi_heatmap)
    axs[0, 1].plot(times, abs_orientations, **extra_kwargs)
    axs[0, 1].set_title('multi heatmap')

    axs[0, 2].pcolormesh(times, angles, dynamic_heatmap)
    axs[0, 2].plot(times, abs_orientations, **extra_kwargs)
    axs[0, 2].set_title('dynamic heatmap')

    axs[0, 0].set_ylabel('angle [deg]')
    axs[-1, 0].set_xlabel('time [s]')
    axs[-1, 1].set_xlabel('time [s]')
    axs[-1, 2].set_xlabel('time [s]')
    axs[0, 0].set_ylim(0, 360)

In [ ]:
import itertools

mic_selection = range(4)

mic_positions = df.iloc[0].mic_positions[mic_selection, :]
beam_former = BeamFormer(mic_positions=mic_positions)

if bin_selection == "fixed":
    selected_idx = [np.argmin(np.abs(df.iloc[0].frequencies - hz)) for hz in selected_hz]
    print(selected_idx)
    frequencies = df.iloc[0].frequencies[selected_idx] 
    beam_former.init_dynamic_estimate(frequencies, combination_n, combination_method, normalization_method)
    beam_former.init_multi_estimate(frequencies, combination_n)
else:
    # TODO(FD): implement good initialization
    pass


R_idx = 0; R_list = [[]] * combination_n
R = None

n_columns = len(columns)
angles = beam_former.theta_scan
times = np.empty(n_columns)
orientations = np.full(n_columns, np.nan)
raw_heatmap = np.zeros((len(angles), n_columns))
dynamic_heatmap = np.zeros((len(angles), n_columns))
multi_heatmap = np.zeros((len(angles), n_columns))

i_col = 0
dynamic_spectrum = None
multi_spectrum = None

with progressbar.ProgressBar(max_value=n_columns, redirect_stdout=True) as p:
    for i in columns:
        row = df.iloc[i]

        if bin_selection == "fixed":
            signals_f = row.signals_f[:, selected_idx] # n_mics x n_freqs
            signals_f = signals_f[mic_selection, :]
            freqs = row.frequencies[selected_idx]
        else:
            # TODO(FD): implement this
            pass

        orientation_deg = row.yaw_deg
        timestamp = row.timestamp

        R_new = beam_former.get_correlation(signals_f.T) # n_freqs x n_mics x n_mics
        if averaging == 'iir':
            if R is None:
                R = R_new
            else:
                R = alpha*R_new + (1-alpha)*R
        elif averaging == 'ma':
            R_list[R_idx] = R_new
            R_idx = (R_idx + 1) % combination_n
            R = np.mean([R_elem for R_elem in R_list if len(R_elem)], axis=0)

        if method == 'mvdr':
            raw_spectrum = beam_former.get_mvdr_spectrum(R, freqs)
        elif method == 'das':
            raw_spectrum = beam_former.get_das_spectrum(R, freqs)
        time_sec = row.audio_timestamp / 1e6

        if not pd.isna(row.yaw_deg):
            #### DYNAMIC
            beam_former.add_to_dynamic_estimates(raw_spectrum, orientation_deg=orientation_deg)
            dynamic_spectrum = beam_former.get_dynamic_estimate()
            dynamic_spectrum = combine_rows(dynamic_spectrum, combination_method, keepdims=True)
            dynamic_spectrum = normalize_rows(dynamic_spectrum, method="zero_to_one")

            #### MULTI
            beam_former.add_to_multi_estimate(signals_f.T, freqs, time_sec, orientation_deg)
            multi_spectrum = beam_former.get_multi_estimate(method=method)
            multi_spectrum = combine_rows(multi_spectrum, combination_method, keepdims=True)
            multi_spectrum = normalize_rows(multi_spectrum, method="zero_to_one")
        else:
            print('not updating dynamic and multi:', i_col)

        #### RAW
        raw_spectrum = combine_rows(raw_spectrum, combination_method, keepdims=True)
        raw_spectrum = normalize_rows(raw_spectrum, method="zero_to_one")

        raw_heatmap[:, i_col] = raw_spectrum.flatten()
        if dynamic_spectrum is not None:
            dynamic_heatmap[:, i_col] = dynamic_spectrum.flatten()
        if multi_spectrum is not None:
            multi_heatmap[:, i_col] = multi_spectrum.flatten()
        times[i_col] = time_sec
        orientations[i_col] = orientation_deg
        i_col += 1

        if i_col >= n_columns:
            break

        p.update(i_col)

    fig, axs = plt.subplots(1, 3, sharey=True, sharex=True)
    plot_results(fig, axs)

# Plotting (can start here directly!)

In [ ]:
from audio_stack.beam_former import BeamFormer

#dict(exp_name = '2020_10_30_dynamic_test', degree=0, motors=''),
#dict(exp_name = '2020_10_30_dynamic_test', degree=90, motors=''),
#dict(exp_name = '2020_10_30_dynamic_move', degree=0, motors='')
#dict(exp_name = '2020_11_03_sweep', degree=90, motors=''),
#dict(exp_name = '2020_11_03_sweep_old', degree=90, motors=''),

plots = [
    dict(exp_name = '2020_11_10_buzzer'),
]

BeamFormer()

for plot_dict in plots:
    exp_name = plot_dict.get('exp_name')
    fname = f'DynamicAnalaysis_{exp_name}.pkl'
    df_results = pd.read_pickle(fname)
    print('read', fname)
    
    df_raw = df_results.loc[(df_results.exp_name==exp_name)]
    for (motors, degree, combination_n, method, averaging), df_chosen in df_raw.groupby(['motors', 'degree', 'combination_n', 'method', 'covariance_averaging']):
        
        assert len(df_chosen) == 1, len(df_chosen)
        
        df_chosen=df_chosen.iloc[0]
        
        if method == 'mvdr':
            continue
        if combination_n == 2:
            continue

        plot_dict['motors'] = motors
        plot_dict['degree'] = degree
        plot_dict['combination_n'] = combination_n
        plot_dict['method'] = method
        plot_dict['averaging'] = averaging

        times = np.arange(df_chosen.raw_heatmap.shape[1])
        angles = BeamFormer.theta_scan * 180 / np.pi

        fig, axs = plt.subplots(1, 3)
        fig.set_size_inches(15, 5)
        axs[0].pcolormesh(times, angles, df_chosen.raw_heatmap)
        axs[0].set_title('raw heatmap')

        axs[1].pcolormesh(times, angles, df_chosen.multi_heatmap)
        axs[1].set_title('multi heatmap')

        axs[2].pcolormesh(times, angles, df_chosen.dynamic_heatmap)
        axs[2].set_title('dynamic heatmap')

        axs[0].set_ylabel('angle [deg]')
        fig.suptitle(plot_dict)